# Day 17: Chronospatial Computer

In [ ]:
import re
import json

with open('data/2024-17-example.txt', 'r') as f:
    test_data = f.readlines()
with open('data/2024-17.txt', 'r') as f:
    data = f.readlines()

test_data = [x.strip() for x in test_data]
data = [x.strip() for x in data]
print('Example data')
print('\n'.join(data))

In [28]:
def prepare_data(input_data: list):
    split_idx = input_data.index('')
    
    # get registers data
    register = {}
    register_data = input_data[:split_idx]
    for line in register_data:
        m = re.match(r'Register (\w+): (\d+)', line)
        register[m.group(1)] = int(m.group(2))
    
    # get instructions
    program = input_data[split_idx+1:][0].replace('Program: ', '')
    instructions = list(map(int, program.split(',')))
    
    # convert instructions to tuples of (opcode, operand)
    instructions = list(zip(instructions[0::2], instructions[1::2]))

    return register, instructions, program


def get_combo_operand(operand: int, register: dict) -> int:
    if operand < 4:
        return operand
    if operand == 4:
        return register['A']
    if operand == 5:
        return register['B']
    if operand == 6:
        return register['C']
    if operand == 7:
        raise ValueError('Invalid combo operand 7')


def compute_instruction(register: dict, opcode: int, operand: int, pointer: int, write_out: list) -> int:
    # adv instruction (div A / 2**combo)
    if opcode == 0:
        register['A'] = register['A'] >> get_combo_operand(operand, register)
    # bxl instruction (bitwise B xor operand)
    elif opcode == 1:
        register['B'] = register['B'] ^ operand
    # bst instruction (combo modulo 8)
    elif opcode == 2:
        register['B'] = get_combo_operand(operand, register) % 8
    # jnz instruction
    elif opcode == 3:
        if register['A'] != 0:
            pointer = (operand - 2)
    # bxc instruction
    elif opcode == 4:
        register['B'] = register['B'] ^ register['C']
    # out instruction
    elif opcode == 5:
        write_out.append(str(get_combo_operand(operand, register) % 8))
    # bdv instruction
    elif opcode == 6:
        register['B'] = register['A'] >> get_combo_operand(operand, register)
    # cdv instruction
    elif opcode == 7:
        register['C'] = register['A'] >> get_combo_operand(operand, register)
    
    return register, pointer, write_out

In [ ]:
# initialize data
register, instructions, _ = prepare_data(data)
pointer = 0
write_out = []
while pointer < (len(instructions)*2):
    opcode, operand = instructions[pointer//2]
    register, pointer, write_out = compute_instruction(register, opcode, operand, pointer, write_out)
    pointer += 2

write_out = ','.join(map(str, write_out)) 
print('Part 1:', write_out)